In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time 
import warnings
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy

In [10]:
df = pd.read_csv("C:/Users/adity/Downloads/applied ai projects/Quora-20220307T081431Z-001/Quora/train.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [11]:
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [12]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [13]:
# merge text
question = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase = False)
tfidf.fit_transform(question)

word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [14]:
#!python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
#!python -m spacy download en

In [15]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_lg')

x = nlp('man')
len(x.vector)

300

In [16]:
vecs = []
for qu2 in tqdm(list(df['question2'])):
    doc = nlp(qu2)
    mean_vec1 = np.zeros((len(doc), len(doc[0].vector)))
    for word2 in doc:
        vec = word2.vector
        #fech df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        # compute final vector
        mean_vec1 += vec*idf
    mean_vec1 = mean_vec1.mean(axis = 0)
    vecs.append(mean_vec1)
df['q2_feats_m'] = list(vecs)

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [1:41:06<00:00, 66.65it/s]


In [17]:
if os.path.isfile('C:/Users/adity/Downloads/applied ai projects/Quora-20220307T081431Z-001/Quora/nlp_features_train.csv'):
    dfnlp = pd.read_csv("C:/Users/adity/Downloads/applied ai projects/Quora-20220307T081431Z-001/Quora/nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('C:/Users/adity/Downloads/applied ai projects/Quora-20220307T081431Z-001/Quora/df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("C:/Users/adity/Downloads/applied ai projects/Quora-20220307T081431Z-001/Quora/df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [20]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
#df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [19]:
df3.head()

,id,q2_feats_m
0,0,"[0.3985789865255356, 13.99160723388195, -0.504..."
1,1,"[4.649687513709068, 9.974927574396133, 20.3301..."
2,2,"[-17.30510464310646, 17.355613683350384, -9.13..."
3,3,"[3.8979105092585087, 2.5458570569753647, -2.05..."
4,4,"[-5.391206279397011, 1.7672210037708282, 1.810..."


In [21]:
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [22]:
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [23]:
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.398579,13.991607,-0.504564,9.254431,13.906436,-4.777694,-5.274421,-0.201208,4.940558,134.735950,...,-17.810438,7.231024,1.531186,-7.528823,0.473802,-11.864658,-11.293788,1.866265,3.616046,11.971096
1,4.649688,9.974928,20.330103,-0.440372,-18.128566,-1.984671,4.906458,-27.797837,-21.262646,96.965297,...,23.015827,3.435464,-5.169600,7.102491,34.516881,6.177686,-27.770856,12.926435,-4.564559,33.919834
2,-17.305105,17.355614,-9.135664,-6.038550,-1.831651,4.547895,17.935764,-4.799029,3.100311,99.380095,...,-24.310109,-1.216773,11.909693,9.591573,11.846737,1.397859,6.454157,-0.271460,-12.500337,27.634567
3,3.897911,2.545857,-2.053792,3.385450,3.424216,-2.282545,-11.763825,6.692485,5.797674,94.978085,...,-5.435584,1.672591,-0.863278,-2.906553,-3.466688,-3.867892,-4.249463,-12.551012,4.494087,-6.223341
4,-5.391206,1.767221,1.810128,-4.097073,-3.623262,8.417368,-25.246265,7.473430,-2.789541,89.594627,...,-10.407441,-8.444207,-14.450059,-12.709382,-4.449050,12.563987,-11.721362,-16.459300,3.626297,-9.790615


In [24]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question2 w2v  dataframe : 300
Number of features in final dataframe  : 329


In [28]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    #df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    #df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df3_q2, on='id',how='left')
    result.to_csv('final_features.csv')